In [1]:
from latex2sympy2 import latex2sympy, latex2latex
import re

In [2]:
tex = r"\frac{\partial}{\partial x}(x^2+y)"
print(latex2sympy(tex))

Derivative(x**2 + y, x)


In [3]:
expresses = {r'18. $\lim _{h \rightarrow 0} \frac{(2+h)^{3}-8}{h}$': (r'\frac{h^3+6h^2+12h+8-8}{h}', r'12')}
#form should be [problem statement]: ([intermediate expression], [solution])

def remove_limits_func(expression):
    # Regular expression to match \lim and remove it for comparison
    return re.sub(r'\\lim _\{.*?\} ', '', expression)

def in_express(sol): # in form [expression1, expression2, ...]
    grade = [False, False]
    for steps in sol:
        step = remove_limits_func(steps)
        print(latex2sympy(step))
        print(latex2sympy(expresses[r'18. $\lim _{h \rightarrow 0} \frac{(2+h)^{3}-8}{h}$'][0]))
        if latex2sympy(step) == latex2sympy(expresses[r'18. $\lim _{h \rightarrow 0} \frac{(2+h)^{3}-8}{h}$'][0]):
            grade[0] = True
            print('We found our step yay! It\'s: solution number ' + str(step) + '.')
            break
        else:
            print('Solution ' + str(step) + ' is not our step.')
    if latex2sympy(sol[-1]) == latex2sympy(expresses[r'18. $\lim _{h \rightarrow 0} \frac{(2+h)^{3}-8}{h}$'][1]):
        print('Final solution is correct')
        grade[1] = True
    else:
        print('YOU\'RE WRONG DON\'T EVER DO MATH AGAIN IN YOUR LIFE')
    return(grade)


student_solution = [r'18. $\lim _{h \rightarrow 0} \frac{(2+h)^{3}-8}{h}$', r'\lim _{h \rightarrow 0} \frac{(h)^{3}+6h^{2}+12h+8-8}{h}', r'\lim _{h \rightarrow 0} h^2+6h+12', r'12']
print(in_express(student_solution))



18
(h**3 + 6*h**2 + 12*h - 8 + 8)/h
Solution 18. $\frac{(2+h)^{3}-8}{h}$ is not our step.
(h**3 + 6*h**2 + 12*h - 8 + 8)/h
(h**3 + 6*h**2 + 12*h - 8 + 8)/h
We found our step yay! It's: solution number \frac{(h)^{3}+6h^{2}+12h+8-8}{h}.
Final solution is correct
[True, True]


In [7]:
from latex2sympy2 import latex2sympy, latex2latex
import re

expresses = {r'18. $\lim _{h \rightarrow 0} \frac{(2+h)^{3}-8}{h}$': (r'\frac{h^3+6h^2+12h+8-8}{h}', r'12')}
#form should be [problem statement]: ([intermediate expression], [solution])

remove_limits = True # controls if limits should be ignored in comparing solution to intermediate step

## TODO: additional forms of solutions, additional things that should be ignored in intermediate step

def limit_remover(expression):
    # Regular expression to match \lim and remove it for comparison
    return re.sub(r'\\lim _\{.*?\} ', '', expression)

def in_express(problem, sol): #problem statement, solution
    grade = [False, False]
    intermediate_step = expresses[problem][0]
    final_solution = expresses[problem][1]
    for steps in sol:
        if remove_limits == True:
            steps = limit_remover(steps)
            intermediate_step = limit_remover(intermediate_step)
        print(latex2sympy(steps))
        print(latex2sympy(intermediate_step))
        if latex2sympy(steps) == latex2sympy(intermediate_step):
            #^ does our step match the intermediate step that we are looking for
            grade[0] = True
            print('We found our step yay! It\'s: solution number ' + str(steps) + '.')
            break #intermediate step is found, no need to try to find it again
        else:
            print('Solution ' + str(steps) + ' is not our step.')
    if latex2sympy(sol[-1]) == latex2sympy(final_solution): # TODO: very possible that final solution isn't just an integer and may need to do the same rules as the intermediate step (remove limits, etc.)
        print('Final solution is correct')
        grade[1] = True
    else:
        print('YOU\'RE WRONG DON\'T EVER DO MATH AGAIN IN YOUR LIFE')
    return(grade)


student_solution = [r'18. $\lim _{h \rightarrow 0} \frac{(2+h)^{3}-8}{h}$', r'\lim _{h \rightarrow 0} \frac{(h)^{3}+6h^{2}+12h+8-8}{h}', r'\lim _{h \rightarrow 0} h^2+6h+12', r'12']
#student solution should be in form [problem statement, step_1, step_2,...] -- each step is an expression
print(in_express(next(iter(expresses)), student_solution))

18
(h**3 + 6*h**2 + 12*h - 8 + 8)/h
Solution 18. $\frac{(2+h)^{3}-8}{h}$ is not our step.
(h**3 + 6*h**2 + 12*h - 8 + 8)/h
(h**3 + 6*h**2 + 12*h - 8 + 8)/h
We found our step yay! It's: solution number \frac{(h)^{3}+6h^{2}+12h+8-8}{h}.
Final solution is correct
[True, True]


In [8]:
import re

def extract_expressions(input_string):
    """
    Extracts all expressions between $ or $$ symbols and further splits each step into individual expressions.
    
    :param input_string: The string containing steps between $ or $$ signs
    :return: A list of individual expressions extracted from each step
    """
    # Regular expression to match content between either $...$ or $$...$$
    step_pattern = re.compile(r'\$(.*?)\$|\$\$(.*?)\$\$')
    
    # Find all steps between $ or $$ signs
    steps = step_pattern.findall(input_string)
    
    # Flatten the list of tuples returned by findall (since it gives two groups, one for $ and another for $$)
    steps = [item for group in steps for item in group if item]

    # Split each step by common expression separators like '=', '+', '-', '*', and '/'
    expressions = []
    for step in steps:
        expressions.extend(re.split(r'[=+\-*/]', step))
    
    # Remove any extra spaces and empty strings
    return [expr.strip() for expr in expressions if expr.strip()]

# Example usage
input_string = "Here is a step: $6(x+4)=6x+24$ and another one $$x^2 - 4 = 0$$."
extracted_expressions = extract_expressions(input_string)
print(extracted_expressions)

'''
def find_student_solution(problem_statement, student_input):
    steps = expression_extractor(student_input)
    for step_num, step in enumerate(steps):
        steps[step_num] = step.split('=')'''
        


['6(x', '4)', '6x', '24']


"\ndef find_student_solution(problem_statement, student_input):\n    steps = expression_extractor(student_input)\n    for step_num, step in enumerate(steps):\n        steps[step_num] = step.split('=')"

In [9]:
import re

def extract_expressions(input_string):
    """
    Extracts all expressions between $ or $$ symbols and further splits each step into individual expressions,
    without breaking inside parentheses or other nested structures.
    
    :param input_string: The string containing steps between $ or $$ signs
    :return: A list of individual expressions extracted from each step
    """
    # Regular expression to match content between either $...$ or $$...$$
    step_pattern = re.compile(r'\$(.*?)\$|\$\$(.*?)\$\$')
    
    # Find all steps between $ or $$ signs
    steps = step_pattern.findall(input_string)
    
    # Flatten the list of tuples returned by findall (since it gives two groups, one for $ and another for $$)
    steps = [item for group in steps for item in group if item]

    # Use a more refined splitting strategy that avoids breaking expressions inside parentheses
    expressions = []
    for step in steps:
        # Match expressions separated by '=' while ignoring contents inside parentheses
        split_exprs = re.split(r'(?<!\([^\)]*)=', step)
        expressions.extend(split_exprs)
    
    # Remove any extra spaces and empty strings
    return [expr.strip() for expr in expressions if expr.strip()]

# Example usage
input_string = "Here is a step: $6(x+4)=6x+24$ and another one $$x^2 - 4 = 0$$."
extracted_expressions = extract_expressions(input_string)
print(extracted_expressions)


error: look-behind requires fixed-width pattern